In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import warnings
warnings.filterwarnings(action='ignore')
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
data = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')

In [ ]:
data.head()

In [ ]:
data.describe(include='all')

In [ ]:
#We will not be doing spatial analysis, so it's best to delete the longitude and latitude columns
data.drop(['longitude','latitude'],axis=1,inplace=True)

In [ ]:
data.info()
#We can see that only total_bedrooms column has null values

In [ ]:
#plotting the numerical columns
data.hist(figsize=(20,15))
#housing_median_age is fairly normal, rest are all right skewed.

In [ ]:
sns.countplot(data['ocean_proximity'])
plt.show()
data['ocean_proximity'].value_counts()

In [ ]:
#checking the significance of each feature using n-way anova
s = data.drop('median_house_value',axis=1)
formula = 'median_house_value~'+"+".join(s.columns)
print(formula)
model = ols(formula, data=data).fit()
aov_table = sm.stats.anova_lm(model)
print(aov_table)
print(aov_table.index[aov_table['PR(>F)']<0.05])

#We reject the null hypothesis for all the columns as p-values are less than 5%.
#Hence, all columns are significant for the regression model.

In [ ]:
data['total_bedrooms'].fillna(data['total_bedrooms'].mean(),inplace=True)

In [ ]:
sns.heatmap(data.corr(),annot=True)

In [ ]:
data = pd.get_dummies(data,drop_first=True)

In [ ]:
X = data.drop('median_house_value',axis=1)
y = data['median_house_value']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
ss = StandardScaler()
X_train_standardised = ss.fit_transform(X_train)
X_train_standardised = pd.DataFrame(X_train_standardised,index=X_train.index,columns=X_train.columns)

In [ ]:
X_test_standardised = ss.transform(X_test)
X_test_standardised = pd.DataFrame(X_test_standardised,index=X_test.index,columns=X_test.columns)

In [ ]:
X_train_standardised.info()

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)

In [ ]:
model.score(X_train,y_train)

In [ ]:
y_predicted = model.predict(X_test)

In [ ]:
model.score(X_test,y_test)